In [ ]:
!pip install -q langchain langchain_google_genai langchain_community langchain_openai textacy nltk spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.6/321.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [ ]:
!pip install -qU transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 21.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login --token xxxxx

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `aijam` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `aijam`


In [ ]:
import pandas as pd
import numpy as np
import re
import os
from typing import Dict, List, Optional, Any, Tuple
from pathlib import Path
import spacy
import spacy.cli
from nltk.tokenize import sent_tokenize
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline as hf_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import unicodedata
import os
import spacy
from typing import Optional, Dict, Any, List
from pathlib import Path
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import download as nltk_download
from transformers import pipeline as hf_pipeline, AutoModelForSequenceClassification, AutoTokenizer
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Set Environment variables
os.environ["OPENAI_API_KEY"] = "put api keys here"
#os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
class TradeAnalyser:


    def __init__(self):
        """
        Initializes the TradeAnalyser with default configurations.
        """
        self.data: Optional[pd.DataFrame] = None
        self.processed_data: Optional[pd.DataFrame] = None
        self.llm_clients: Dict[str, Any] = {}
        self.hf_pipelines: Dict[str, hf_pipeline] = {}
        self.classification_prompts: Dict[str, Dict[str, Any]] = self._initialize_standardized_prompts()

        # Sentiment-related attributes
        self.pos_words: Optional[set] = None
        self.neg_words: Optional[set] = None
        self.pos_pattern: Optional[str] = None
        self.neg_pattern: Optional[str] = None
        self.negation_set: Optional[set] = None
        self.lemmatizer: Optional[WordNetLemmatizer] = None
        self.stopw: Optional[set] = None


        # Load spaCy model for sentence splitting if available
        try:
            self.spacy_nlp = spacy.load("en_core_web_sm")
        except:
            print("Downloading 'en_core_web_sm' model for spaCy...")
            spacy.cli.download("en_core_web_sm")
            self.spacy_nlp = spacy.load("en_core_web_sm")

    def _initialize_standardized_prompts(self) -> Dict[str, Dict[str, Any]]:
        """
        Initializes all classification prompts with a standardized XML-tagged output format.
        Returns:
            Dict: A nested dictionary of classification tasks and their prompt variants.
        """
        return {
            "sentiment_analysis": {
                "basic": PromptTemplate.from_template(
                    """As an economist and trade/industrial policies expert at the International Monetary Fund (IMF), your task is to analyze and classify the sentiment of sentences extracted from news articles related to the International Monetary Fund (IMF).

                    Provide your analysis in the following XML format. A tone_score of -5.0 is strongly negative, 5.0 is strongly positive, and 0.0 is neutral.

                    Always give your answer in english even if the text is in spanish or french or others languages.

                    <sentiment_label>strong negative/moderate negative/neutral/moderate positive/strong positive</sentiment_label>
                    <tone_score>-5.0 to 5.0</tone_score>
                    <justification>Brief explanation of key phrases that influenced your decision</justification>

                    Classify this text now :

                    "{text}"
                    """
                ),
                "chain_of_thought": PromptTemplate.from_template(
                    """As an economist and expert in Trade at the IMF, your task is to analyze and classify the sentiment of sentences extracted from news articles related to the International Monetary Fund (IMF). Always give your answer in english even if the text is in spanish or french or others languages.
                    Think step by step to determine the sentiment:
                    1. Identify key trade and industrial policies terms and their connotations.
                    2. Consider the overall economic and trade or broad impact described.
                    3. Evaluate the tone toward economic policies/outcomes.
                    Provide your analysis in the following XML format. A tone_score of -5.0 is strongly negative, 5.0 is strongly positive, and 0.0 is neutral.
                    <sentiment_label>strong negative/moderate negative/neutral/moderate positive/strong positive</sentiment_label>
                    <tone_score>-5.0 to 5.0</tone_score>
                    <justification>Brief explanation of key phrases that influenced your decision</justification>

                    Classify this text now :

                    "{text}"
                    """
                ),
                "few_shot": FewShotPromptTemplate(
                    examples=[
                        {
                            "text": "The government announced a new export subsidy program for strategic industries to boost competitiveness and support small exporters.",
                            "analysis": "<sentiment_label>moderate positive</sentiment_label><tone_score>3</tone_score><justification>The phrase 'boost competitiveness' and 'support small exporters' reflects a constructive policy measure intended to strengthen trade performance, suggesting a moderately positive outlook on industrial strategy.</justification>"
                        },
                         {
                            "text": "To protect domestic producers, the government imposed anti-dumping duties on low-cost steel imports from several countries.",
                            "analysis": "<sentiment_label>moderate positive</sentiment_label><tone_score>2</tone_score><justification>'Protect domestic producers' suggests a defensive but supportive policy stance, reflecting moderate optimism for local industry resilience.</justification>"
                        },
                        {
                            "text": "Several business groups warned that the new import ban on machinery parts will severely disrupt domestic production and raise costs.",
                            "analysis": "<sentiment_label>strong negative</sentiment_label><tone_score>-4</tone_score><justification>'Warned', 'severely disrupt', and 'raise costs' indicate a strongly negative reaction to the trade policy, highlighting its damaging economic implications.</justification>"
                        }
                    ],
                    example_prompt=PromptTemplate.from_template("Sentence: {text}\nAnalysis: {analysis}\n"),
                    prefix="""As an economist and expert at the IMF, your task is to analyze and classify the sentiment of sentences extracted from news articles related to the International Monetary Fund (IMF).Always give your answer in english even if the text is in spanish or french or others languages.\n\nProvide your analysis in the following XML format.
                    A tone_score of -5.0 is strongly negative, 5.0 is strongly positive, and 0.0 is neutral.
                    <sentiment_label>strong negative/moderate negative/neutral/moderate positive/strong positive</sentiment_label>
                    <tone_score>-5.0 to 5.0</tone_score>
                    <justification>Brief explanation of key phrases that influenced your decision</justification> """,
                    suffix="Sentence: {text}\nAnalysis:",
                    input_variables=["text"]
                )
            },
            "tense_extraction": {
                "basic": PromptTemplate.from_template(
                    """You are an economist and trade/industrial policies expert at the International Monetary Fund (IMF). Always give your answer in english even if the text is in spanish or french or others languages.
          Your task is to determine the temporal context of economic statements.
          Each sentence provided must be assessed to identify whether it refers to the **past**, **present**, or **future**.
          Your analysis must be precise, context-aware, and linguistically accurate, with particular attention to verb tense and economic context.
          Mistakes are not acceptable, as your output may inform high-level policy evaluation.

          You will receive a single sentence. Read it carefully and return the appropriate temporal classification.
          Format your response using XML tags, as shown below:

          <tense>one of Past/Present/Future</tense>

          Extract tense from this text now :
          "{text}"
          """
                          )
            },
            "product_categories": {
                "basic": PromptTemplate.from_template(
                    """You are an economist and trade/industrial policies expert at the International Monetary Fund (IMF). Always give your answer in english even if the text is in spanish or french or others languages.
          Your task is to determine the type of products that are targeted by policy interventions.
          Please read each sentence carefully and determine the most appropriate product category based on its content. The categories to consider include:

          * Medical Products: Includes medical consumables, vaccines, medicines, and medical equipment.

          * Military/Civilian Dual-Use Products: Items usable for both civilian and military applications. This includes certain machinery, hydrogen, or aerospace components used in both domains.

          * Low-Carbon Technology: Includes wind turbines, solar panels, biomass systems, carbon capture equipment, and other technologies contributing to the low-carbon transition.

          * Semiconductors: Semiconductors and related materials or technologies that enable semiconductor development.

          * Advanced Technology Products: Products that rely on cutting-edge technologies, including aerospace, opto-electronics, robotics, quantum, and nuclear technologies.

          * Critical Minerals: Minerals essential for manufacturing modern technology (e.g., lithium, cobalt, rare earths).

          * Steel and Aluminum: Includes crude or processed steel, iron, and aluminum products.

          * Chemicals: Includes basic and specialty chemicals such as petrochemicals, fertilizers, industrial compounds, and plastics.

          * Other: Use this when the sentence does not clearly fall into the categories above.

          It's possible to have many product categories in a single sentence. So you can separate them with a comma.

          Format your response using XML tags, as shown below:

          <result>one of Low-Carbon Technology/Dual-use products/Critical minerals/Advanced technology products/Other</result>

          Extract type of products from this text now :
          "{text}"
          """)
            },
            "stated_motive": {
                "basic":PromptTemplate.from_template(
    """You are an economist and trade expert at the International Monetary Fund (IMF). Always give your answer in English, even if the text is in Spanish, French, or other languages.

    Your task is to identify the stated motive(s) for a trade or industrial policy intervention, using the exact classification used in the NIPO database and related figures. Read the text carefully and assign one or more of the following motives:

    * Strategic competitiveness: The policy supports the competitiveness, innovation, or global positioning of domestic firms in strategic sectors.

    * Climate change mitigation: The policy aims to reduce greenhouse gas emissions, support renewable energy, or promote the green transition.

    * Resilience/security of supply (non-food): The policy seeks to ensure stable domestic supply chains or reduce dependence on foreign suppliers for non-food products.

    * National security: The policy is intended to protect national defense interests or mitigate military-related threats.

    * Geopolitical concerns: The policy responds to international tensions, sanctions, or rivalry with specific countries or blocs.

    * Food security: The policy ensures domestic food availability, affordability, or agricultural self-sufficiency.

    * Public health concerns: The policy addresses health risks, pandemics, or ensures access to critical medicines and health equipment.

    * Digital transformation: The policy supports the development, adoption, or protection of digital infrastructure, services, or technologies.

    * Other: The motive is explicitly stated but does not match the above categories (e.g., employment generation, regional development).

    Multiple motives can be present in a single sentence. Separate multiple motives with commas.

    Format your response using XML tags, as shown below:

    <result>one or more of National security/Geopolitical concerns/Resilience/security of supply (non-food)/Domestic competitiveness in strategic sectors/Climate change mitigation and other environmental objectives/None</result>

    Extract the stated motive(s) from this text now:
    "{text}"
    """)
            },

            "broad_categories": {
                "basic": PromptTemplate.from_template(
    """You are an economist and trade expert at the International Monetary Fund (IMF). Always give your answer in English, even if the text is in Spanish, French, or other languages.

    Your task is to identify the broad category (or categories) of policy instruments being applied. Carefully read the sentence and determine which of the following categories best describe the type of policy intervention:

    * Export barriers: Measures that restrict exports, including export bans, tariffs, quotas, licensing requirements, and other export-related trade barriers.

    * Import barriers: Measures that restrict imports, including import bans, tariffs, quotas, licensing requirements, and other import-related trade barriers.

    * Domestic subsidies: Financial or fiscal support provided to domestic producers, including tax rebates, grants, state loans, loan guarantees, price stabilization, or production subsidies.

    * Export incentives: Measures that promote exports through financial or fiscal means, such as export subsidies, tax incentives, trade financing, or loans for exporters.

    * Foreign Direct Investment measures: Measures affecting foreign ownership or investment, including entry and ownership restrictions and FDI screening decisions.

    * Procurement policies: Policies or laws that affect public procurement and may favor domestic firms or production.

    * Localization incentives or requirements: Measures that require or promote local production or content, including local content requirements and public procurement localization measures.

    * Other: If none of the above categories are clearly mentioned, return 'Other'.

    More than one category may apply to a single sentence. If so, separate them with commas.

    Format your response using XML tags, as shown below:

    <result>one or more of Export barriers/Import barriers/Domestic subsidies/Export incentives/Foreign Direct Investment measures/Procurement policies/Localization incentives or requirements/None</result>

    Extract the broad policy instrument category (or categories) from this text now:
    "{text}"
    """
)
                },
            "measure_nature": {
                "basic":PromptTemplate.from_template(
    """You are a trade policy analyst at the IMF. Your task is to assess the nature of a trade or industrial policy measure. Please classify the measure into one of the following categories:

- Liberalising: The measure facilitates or promotes international trade by reducing restrictions, increasing transparency, or encouraging imports and exports. Examples include tariff reductions, elimination of quotas, removal of export bans, simplification of import licensing, or other steps that enhance openness to global markets.

- Distortive: The measure restricts trade or gives preferential treatment to domestic actors in a way that distorts competition. This includes export bans or taxes, import tariffs or quotas, local content requirements, subsidies (such as production, export, or investment subsidies), financial assistance, bailouts, price controls, and public procurement preferences aimed at protecting local firms.

- Other: The measure does not clearly fall under liberalising or distortive (e.g., administrative updates, procedural changes, or unclear intent), or its economic impact is uncertain based on the available information.

Please read the sentence carefully and return the most appropriate category.

Format your answer in this XML structure:
<result>Distortive</result>

Now classify this policy measure:
"{text}"
"""
)
            },
            "timeline":{
                "basic": PromptTemplate.from_template(
    """You are a trade policy analyst at the IMF. Your task is to extract the key timeline information associated with a trade or industrial policy measure.

For each of the following elements, extract the most relevant time reference **as free text** (e.g., "January 2024", "Q2 2025", "early 2023"). Dates do not need to follow a strict format.

Definitions:
- Announcement date: When the measure was officially communicated to the public.
- Implementation date: When the measure came into force or was applied in practice.
- Removal date: When the measure was officially revoked, expired, or ended.

If the text **does not mention** the timing clearly or **does not describe** a concrete policy measure with a timeline, label the item as:
**"Not applicable"**

Return your answer in the XML format below:

<result>
  <announcement_date>Free text or Not applicable</announcement_date>
  <implementation_date>Free text or Not applicable</implementation_date>
  <removal_date>Free text or Not applicable</removal_date>
</result>

Now extract the timeline information from this text:
"{text}"
"""
)

            },
            "intervention_type":{
                "basic":PromptTemplate.from_template(
    """You are a trade policy expert at the IMF. Your task is to identify the most relevant type(s) of trade or industrial policy intervention described in the sentence below.

Please choose from the following comprehensive list of intervention types (select one or more, separated by commas if needed):

Financial grant, State loan, Import tariff, Trade finance, Local content incentive, Anti-dumping, Loan guarantee, Tax or social insurance relief, State aid, unspecified, Financial assistance in foreign market, Public procurement localisation, Export tax, Capital injection and equity stakes (including bailouts), Import tariff quota, Local value added incentive, Production subsidy, Price stabilisation, Export ban, FDI: Entry and ownership rule, Import price benchmark, Internal taxation of imports, Import licensing requirement, Controls on commercial transactions and investment instruments, Export licensing requirement, Import ban, Labour market access, Anti-subsidy, Other import charges, Import-related non-tariff measure, nes, Export quota, FDI: Treatment and operations, nes, Interest payment subsidy, Tax-based export incentive, Safeguard, Import quota, Other export incentive, Local content requirement, Public procurement, nes, FDI: Financial incentive, Anti-circumvention, Instrument unclear, Public procurement preference margin, Export-related non-tariff measure, nes, Export subsidy, In-kind grant, Public procurement access, Local operations requirement, State aid, nes, Localisation, nes, Local labour requirement, Import incentive, Trade payment measure, Controls on credit operations, Post-migration treatment, Control on personal transactions, Local supply requirement for exports, Export tariff quota, Foreign customer limit, Import monitoring, Special safeguard, Repatriation & surrender requirements, Local labour incentive, Intellectual property protection, Local operations incentive, Minimum import price, Competitive devaluation, Export price benchmark, Technical barrier to trade, Trade balancing measure, Local value added requirement, Other

Return the most appropriate intervention type(s) using this XML format. If there are multiple types, separate them by commas:

Example : <result>Export subsidy, State loan</result>

Now classify the intervention type(s) in the following text:
"{text}"
"""
)
}
        }

    def load_llm_client(self, client_name: str, model_id: str, **kwargs) -> None:
        """
        Configures and loads a large language model client (e.g., OpenAI, Google).
        Args:
            client_name (str): A unique name for the client (e.g., 'gpt', 'gemini').
            model_id (str): The specific model to use (e.g., 'gpt-4o', 'gemini-pro').
            **kwargs: Additional keyword arguments for the LLM client constructor.
        Raises:
            ValueError: If the client_name is not recognized.
        """
        print(f"⚙️ Loading LLM client: {client_name} with model '{model_id}'...")
        if client_name == "openai":
            api_key = os.getenv("OPENAI_API_KEY")
            if not api_key:
                raise ValueError("OPENAI_API_KEY environment variable not set.")
            self.llm_clients[client_name] = ChatOpenAI(model=model_id, api_key=api_key, **kwargs)
        elif client_name == "google":
            api_key = os.getenv("GOOGLE_API_KEY")
            if not api_key:
                raise ValueError("GOOGLE_API_KEY environment variable not set.")
            self.llm_clients[client_name] = ChatGoogleGenerativeAI(model=model_id, google_api_key=api_key, **kwargs)
        else:
            raise ValueError(f"LLM client '{client_name}' not supported. Choose 'openai' or 'google'.")
        print(f"LLM client '{client_name}' loaded successfully.")

    def load_huggingface_pipeline(self, pipeline_name: str, model_id: str, task: str, **kwargs) -> None:
        """
        Configures and loads a Hugging Face pipeline for specific tasks like sentiment analysis.
        Args:
            pipeline_name (str): A unique name for the pipeline (e.g., 'finbert_sentiment').
            model_id (str): The Hugging Face model ID (e.g., 'yiyanggu/finbert-tone').
            task (str): The Hugging Face task name (e.g., 'sentiment-analysis', 'text-classification').
            **kwargs: Additional keyword arguments for the pipeline constructor.

        Raises:
            ValueError: If the pipeline task is not recognized.
        """
        print(f"Loading Hugging Face pipeline: {pipeline_name} with model '{model_id}' for task '{task}'...")
        if task not in ['sentiment-analysis', 'text-classification']:
            raise ValueError(f"Hugging Face task '{task}' not supported.")

        try:
            print("Pipeline is", pipeline_name)
            if "modernfinbert" in pipeline_name:
              model = AutoModelForSequenceClassification.from_pretrained('beethogedeon/Modern-FinBERT-large', num_labels=3)
              tokenizer = AutoTokenizer.from_pretrained('answerdotai/ModernBERT-large')
            elif "finbert" in pipeline_name:
              model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
              tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
            else:
              tokenizer = AutoTokenizer.from_pretrained(model_id)
              model = AutoModelForSequenceClassification.from_pretrained(model_id)

            self.hf_pipelines[pipeline_name] = hf_pipeline(task=task, model=model, tokenizer=tokenizer, device_map="auto",**kwargs)
            print(f"Hugging Face pipeline '{pipeline_name}' loaded successfully.")
        except Exception as e:
            print(f"Failed to load Hugging Face pipeline '{pipeline_name}'. Error: {e}")
            raise

    def load_sentiment_dictionary(self, csv_path: str) -> None:
        """
        Loads a sentiment lexicon from a CSV file with 'pos' and 'neg' columns
        and initializes sentiment analysis resources.
        Args:
            csv_path (str): Path to the CSV file containing positive and negative words.
        Raises:
            FileNotFoundError: If the CSV file is not found.
            ValueError: If the CSV file does not contain 'pos' and 'neg' columns.
        """
        print(f"Loading sentiment dictionary from '{csv_path}'...")
        if not os.path.exists(csv_path):
            raise FileNotFoundError(f"Sentiment dictionary file not found at: {csv_path}")

        try:
            lexicon_df = pd.read_csv(csv_path)
            if 'pos' not in lexicon_df.columns or 'neg' not in lexicon_df.columns:
                raise ValueError("CSV must contain 'pos' and 'neg' columns.")

            self.pos_words = set(lexicon_df['pos'].dropna().str.lower())
            self.neg_words = set(lexicon_df['neg'].dropna().str.lower())

            # Compile regex patterns for faster searching
            self.pos_pattern = r'\b(' + '|'.join(re.escape(w) for w in self.pos_words) + r')\b' if self.pos_words else ''
            self.neg_pattern = r'\b(' + '|'.join(re.escape(w) for w in self.neg_words) + r')\b' if self.neg_words else ''

            # Initialize NLTK components for sentiment analysis
            self.lemmatizer = WordNetLemmatizer()
            self.stopw = set(stopwords.words('english'))
            self.negation_set = {'no', 'not', 'never', 'none', 'neither', 'nor', 'don\'t', 'doesn\'t', 'didn\'t',
                                 'couldn\'t', 'wouldn\'t', 'shouldn\'t', 'won\'t', 'can\'t'}

            print(f"Sentiment dictionary loaded: {len(self.pos_words)} positive words, {len(self.neg_words)} negative words.")
        except Exception as e:
            print(f"Failed to load sentiment dictionary. Error: {e}")
            raise

    def load_data(self, file_path: str, text_column: str) -> None:
        """
        Loads data from a specified file (CSV, Pickle, etc.) and stores it in the instance.
        Args:
            file_path (str): The path to the input data file.
            Supported formats: .csv, .pkl, .pickle.
            text_column (str): The name of the column containing the raw text to be analyzed.
        Raises:
            FileNotFoundError: If the specified file does not exist.
            ValueError: If the file format is not supported or the text column is not found.
        """
        print(f"Loading data from {file_path}...")
        if not Path(file_path).is_file():
            raise FileNotFoundError(f"File not found: {file_path}")

        file_extension = Path(file_path).suffix.lower()

        if file_extension == '.csv':
            self.data = pd.read_csv(file_path)
        elif file_extension in ['.pkl', '.pickle']:
            self.data = pd.read_pickle(file_path)
        else:
            raise ValueError(f"Unsupported file format: {file_extension}. Supported formats are .csv, .pkl, and .pickle.")

        if text_column not in self.data.columns:
            raise ValueError(f"Text column '{text_column}' not found in the DataFrame.")

        self.processed_data = self.data.copy()
        print(f"Data loaded. Shape: {self.data.shape}")

    def split_into_sentences(self, text_column: str, strategy: str = 'nltk') -> None:
        """
        Splits the text in the specified column into individual sentences.
        The sentences are stored in a new DataFrame, with each row representing a sentence.
        Args:
            text_column (str): The name of the column containing the text to be split.
            strategy (str): The splitting strategy to use ('nltk', 'spacy', 'langchain').
        Raises:
            ValueError: If no data is loaded or an invalid strategy is provided.
        """
        if self.processed_data is None:
            raise ValueError("No data to process. Please run `load_data` first.")

        print(f"Splitting text into sentences using '{strategy}' strategy...")

        sentences = []
        # We need to iterate over the *original* loaded data's text column to ensure we split full articles
        # that have passed initial filtering, rather than an already split sentence-level dataframe.
        # This assumes `self.data` holds the original articles and `self.processed_data` is used for sentence-level analysis.
        # Given the refactoring, `self.processed_data` will now contain the filtered *documents* initially.
        for doc_id, text in self.processed_data[text_column].items():
            if pd.isna(text):
                continue

            if strategy == 'nltk':
                # NLTK's sentence tokenizer
                sents = sent_tokenize(text)
            elif strategy == 'spacy':
                # spaCy's sentence splitter
                sents = [sent.text for sent in self.spacy_nlp(text).sents]
            elif strategy == 'langchain':
                # LangChain's RecursiveCharacterTextSplitter
                splitter = RecursiveCharacterTextSplitter(
                    chunk_size=512,  # A reasonable chunk size for a sentence-like split
                    chunk_overlap=0,
                    separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""]
                )
                sents = splitter.split_text(text)
            else:
                raise ValueError(f"Invalid splitting strategy: '{strategy}'. Choose 'nltk', 'spacy', or 'langchain'.")

            for sentence_text in sents:
                sentences.append({'doc_id': doc_id, 'sentence_text': sentence_text.strip()})

        # Create a new DataFrame from the sentences, overwriting processed_data for sentence-level tasks
        self.processed_data = pd.DataFrame(sentences)
        print(f"Text splitting completed. Total sentences: {len(self.processed_data)}")

    def _parse_xml_output(self, text: str, tags: List[str]) -> Dict[str, Any]:
        """
        Extracts specified XML-like tags from a text, whether or not it includes markdown-style
        XML code blocks (e.g., ```xml ... ```). All markdown syntax is stripped, and the
        remaining XML content is parsed.

        Args:
            text (str): The raw text which may contain XML content directly or within markdown code blocks.
            tags (List[str]): A list of XML tag names to extract (e.g., ['label', 'score']).

        Returns:
            Dict[str, Any]: A dictionary mapping each tag to its extracted content. If a value can be
                            converted to float (e.g., for *_score tags), it will be. If a tag is not
                            found, its value will be None.
        """
        results: Dict[str, Any] = {}

        # Step 1: Remove all ```xml ... ``` blocks and extract their content
        xml_blocks = re.findall(r"```xml\s*(.*?)```", text, re.DOTALL | re.IGNORECASE)
        markdown_xml = "\n".join(block.strip() for block in xml_blocks)

        # Step 2: Remove markdown xml blocks from text to avoid duplication
        cleaned_text = re.sub(r"```xml\s*.*?```", "", text, flags=re.DOTALL | re.IGNORECASE)

        # Step 3: Combine extracted XML from markdown with any raw inline XML
        combined_xml = markdown_xml + "\n" + cleaned_text.strip()

        # Step 4: Extract specified tags from the combined XML content
        for tag in tags:
            match = re.search(f"<{tag}>(.*?)</{tag}>", combined_xml, re.DOTALL)
            if match:
                value = match.group(1).strip()
                try:
                    results[tag] = float(value) if tag.endswith('_score') else value
                except ValueError:
                    results[tag] = value
            else:
                results[tag] = None

        return results

    def run_classification_task(self, task_name: str, client_name: str, prompt_type: str, text_column: str, save_path: Optional[str] = None) -> None:
        """
        Runs a classification task using a configured LLM or Hugging Face pipeline.
        This method applies the chosen prompt and model to the specified text column and
        adds the results to the processed data DataFrame.
        It handles different client types
        and standardizes the output column names.
        Args:
            task_name (str): The name of the classification task (e.g., 'sentiment_analysis').
            client_name (str): The name of the configured client ('gpt', 'gemini', 'finbert_sentiment').
            prompt_type (str): The prompt variant to use ('basic', 'chain_of_thought', 'few_shot').
            text_column (str): The column containing the text to classify.
            save_path (str, optional): Path to save the processed data after this task.
        Raises:
            ValueError: If no processed data is found or the client/task is not configured.
        """
        if self.processed_data is None:
            raise ValueError("No processed data found. Run `load_data` and relevant filtering steps first.")

        is_llm_client = client_name in self.llm_clients
        is_hf_pipeline = client_name in self.hf_pipelines

        if not is_llm_client and not is_hf_pipeline:
            raise ValueError(f"Client '{client_name}' not configured. Please use `load_llm_client` or `load_huggingface_pipeline`.")

        print(f"Running '{task_name}' with '{client_name}' ({prompt_type})...")

        if is_llm_client:
            llm_client = self.llm_clients[client_name]
            try:
                prompt_template = self.classification_prompts[task_name][prompt_type]
            except KeyError:
                raise ValueError(f"Prompt type '{prompt_type}' not available for task '{task_name}'.")

            chain = prompt_template | llm_client | StrOutputParser()

            if task_name == 'sentiment_analysis':
                tags_to_extract = ['sentiment_label', 'tone_score', 'justification']
            elif task_name == 'product_categories':
                tags_to_extract = ['result']
            elif task_name == 'stated_motive':
                tags_to_extract = ['result']
            elif task_name == 'broad_categories':
                tags_to_extract = ['result']
            elif task_name == 'timeline':
                tags_to_extract = ['announcement_date', 'implementation_date', 'removal_date']
            elif task_name == 'measure_nature':
                tags_to_extract = ['result']
            elif task_name == 'trade_type':
                tags_to_extract = ['result']
            elif task_name == 'tense_extraction':
                tags_to_extract = ['tense']
            else:
                tags_to_extract = [f'{task_name}_label', 'justification']

            results = []
            for idx, text in self.processed_data[text_column].items():
                raw_output = chain.invoke({"text": text})
                print(raw_output)

                parsed_output = self._parse_xml_output(raw_output, tags_to_extract)
                print(parsed_output)
                results.append(parsed_output)

            results_df = pd.DataFrame(results)
            llm_short_name = client_name
            task_short_name = task_name.split('_')[0]

            for tag in tags_to_extract:
                if tag in results_df.columns:
                    col_name = f"{llm_short_name}_{tag}_{prompt_type}" if 'label' in tag or 'score' in tag or tag == 'reform' or tag == 'stakeholders' or tag == 'policy_area' or tag == 'tense' or tag== 'reform_stage' else f"{llm_short_name}_{task_short_name}_{tag}_{prompt_type}"
                    self.processed_data[col_name] = results_df[tag]

        elif is_hf_pipeline:
            hf_pipeline_client = self.hf_pipelines[client_name]

            hf_results = hf_pipeline_client(self.processed_data[text_column].tolist())

            labels = [res['label'].capitalize() for res in hf_results]
            scores = [res['score'] for res in hf_results]

            model_short = client_name.split('_')[0]
            task_short = task_name.split('_')[0]

            score_map = {'positive': 1, 'neutral': 0, 'negative': -1}

            self.processed_data[f'{model_short}_{task_short}_label'] = labels
            self.processed_data[f'{model_short}_{task_short}_score'] = scores

            if "finbert" in model_short:

              self.processed_data[f'{model_short}_{task_short}_label'] = self.processed_data[f'{model_short}_{task_short}_label'].str.lower()
              self.processed_data[f'{model_short}_{task_short}_score'] = self.processed_data[f'{model_short}_{task_short}_score'] * self.processed_data[f'{model_short}_{task_short}_label'].map(score_map)
              self.processed_data.drop(columns=[f'{model_short}_{task_short}_label'], inplace=True)

        print(f"Task '{task_name}' completed. Results added to DataFrame.")

        if save_path:
            self.save_processed_data(save_path)

    def _clean_build(self, text: str) -> str:
        text = text.lower()
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', '', text)
        text = re.sub('\w*\d\w*', '', text)
        return text

    def _normalize_text(self, text: str) -> str:
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return text

    def _get_words_to_check_negation(self, word_list: List[str]) -> List[str]:
        words_to_check = []
        for i, word in enumerate(word_list):
            if word in self.negation_set:
                if i + 1 < len(word_list):
                    words_to_check.append(word_list[i+1])
                if i + 2 < len(word_list):
                    words_to_check.append(word_list[i+2])
        return words_to_check

    def _count_sentiment_words_with_negation(self, text: str) -> Dict[str, int]:
        if not all([self.pos_words, self.neg_words, self.negation_set, self.lemmatizer, self.stopw]):
            raise ValueError("Sentiment lexicon and NLTK resources not loaded. Call `load_sentiment_dictionary` first.")

        cleaned_text = self._normalize_text(self._clean_build(text))
        words = word_tokenize(cleaned_text)
        words = [self.lemmatizer.lemmatize(word) for word in words if word not in self.stopw]

        pos_count = 0
        neg_count = 0

        negated_words = self._get_words_to_check_negation(words)

        for word in words:
            if word in self.pos_words and word not in negated_words:
                pos_count += 1
            elif word in self.neg_words and word not in negated_words:
                neg_count += 1
            elif word in self.pos_words and word in negated_words:
                neg_count += 1  # Negated positive word becomes negative
            elif word in self.neg_words and word in negated_words:
                pos_count += 1  # Negated negative word becomes positive

        return {'pos_words_count': pos_count, 'neg_words_count': neg_count}

    def _count_number_of_tokens(self, text: str) -> int:
        cleaned_text = self._normalize_text(self._clean_build(text))
        return len(word_tokenize(cleaned_text))

    def run_dictionary_sentiment(self, text_column: str, save_path: Optional[Path] = None) -> None:
        """
        Runs dictionary-based sentiment analysis on the specified text column, adding new columns
        for positive word count, negative word count, total tokens, and sentiment score.
        Args:
            text_column (str): The name of the column containing the text to analyze.
            save_path (Optional[Path]): If provided, the processed data will be saved to this path.
        Raises:
            ValueError: If processed_data is not loaded or sentiment dictionary is not loaded.
        """
        if self.processed_data is None:
            raise ValueError("No data to process. Please run `load_data` and `split_into_sentences` first.")
        if not all([self.pos_words, self.neg_words]):
            raise ValueError("Sentiment dictionary not loaded. Call `load_sentiment_dictionary` first.")

        print("Running dictionary-based sentiment analysis...")

        # Apply sentiment counting
        sentiment_results = self.processed_data[text_column].apply(lambda x: self._count_sentiment_words_with_negation(str(x)) if pd.notna(x) else {'pos_words_count': 0, 'neg_words_count': 0})
        self.processed_data['pos_words_count'] = sentiment_results.apply(lambda x: x['pos_words_count'])
        self.processed_data['neg_words_count'] = sentiment_results.apply(lambda x: x['neg_words_count'])

        # Apply token counting
        self.processed_data['total_tokens'] = self.processed_data[text_column].apply(lambda x: self._count_number_of_tokens(str(x)) if pd.notna(x) else 0)

        # Calculate sentiment score, handling division by zero
        self.processed_data['sentiment_score'] = np.where(
            self.processed_data['total_tokens'] > 0,
            (self.processed_data['pos_words_count'] - self.processed_data['neg_words_count']) / self.processed_data['total_tokens'],
            0
        )

        print("Dictionary-based sentiment analysis completed.")

        if save_path:
            self.processed_data.to_pickle(save_path)
            print(f"Processed data with sentiment scores saved to {save_path}")

    def get_processed_data(self) -> pd.DataFrame:
        """
        Returns the DataFrame with processed data and classification results.
        Returns:
            pd.DataFrame: The DataFrame containing all analyzed sentences and their results.
        Raises:
            ValueError: If no processed data is available.
        """
        if self.processed_data is None:
            raise ValueError("No processed data available. Run the pipeline steps first.")
        return self.processed_data

    def save_processed_data(self, file_path: str) -> None:
        """
        Saves the processed data to a pickle file.
        Args:
            file_path (str): The path to save the pickle file.
        Raises:
            ValueError: If no processed data is available.
        """
        if self.processed_data is None:
            raise ValueError("No processed data available. Run the pipeline steps first.")
        self.processed_data.to_pickle(file_path)
        print(f"Data saved to '{file_path}'.")

    def run_full_pipeline(self, file_path: str, text_column: str, llm_client_name: str, llm_model_id: str,
                          sentiment_csv_path: Optional[str] = None, save_dir: str = '.',
                          initial_save_name: str = 'filtered_by_keywords.pkl',
                          llm_filtered_save_name: str = 'filtered_by_llm.pkl',
                          sentence_split_save_name: str = 'sentences_processed.pkl',
                          final_save_name: str = 'final_analysis_results.pkl'):
        """
        Runs the full analysis pipeline with filtering steps as requested.

        Args:
            file_path (str): Path to the initial data file (CSV, Pickle).
            text_column (str): Name of the column containing the main text for analysis.
            llm_client_name (str): Name of the LLM client to use for reform classification (e.g., 'google', 'openai').
            llm_model_id (str): Model ID for the LLM (e.g., 'gemini-pro', 'gpt-4o').
            sentiment_csv_path (str, optional): Path to the sentiment dictionary CSV.
            save_dir (str): Directory to save intermediate and final processed data.
            initial_save_name (str): Filename for data after keyword filtering.
            llm_filtered_save_name (str): Filename for data after LLM reform classification.
            sentence_split_save_name (str): Filename for data after sentence splitting.
            final_save_name (str): Filename for the final processed data.
        """
        os.makedirs(save_dir, exist_ok=True)
        print("Starting the Reform Analysis Pipeline...")

        # 1. Load data
        self.load_data(file_path, text_column)
        print(f"Initial data loaded. Number of documents: {len(self.processed_data)}")

        # Split into sentences
        self.split_into_sentences(text_column=text_column, strategy='spacy') # Or 'spacy', 'langchain'

        #self.processed_data = initial_filtered_data # Update processed_data to contain only keyword-filtered documents
        self.save_processed_data(Path(save_dir) / initial_save_name)
        print(f"Step 1: Documents filtered by keywords. {len(self.processed_data)} documents contains reforms keywords. Saved to {Path(save_dir) / initial_save_name}")

        # 3. Second verification with LLM-based reform classification
        print("Step 2: Classifying reforms using LLM (reform_classification)...")
        self.load_llm_client(llm_client_name, llm_model_id, temperature=0.1)

        # Sentiment Analysis (LLM)
        self.run_classification_task(
            task_name='sentiment_analysis',
            client_name=llm_client_name,
            prompt_type='basic', # or 'chain_of_thought', 'few_shot'
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name # Save after each major task group
        )

        self.run_classification_task(
            task_name='sentiment_analysis',
            client_name=llm_client_name,
            prompt_type='chain_of_thought',#, 'few_shot'
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name # Save after each major task group
        )

        self.run_classification_task(
            task_name='sentiment_analysis',
            client_name=llm_client_name,
            prompt_type='few_shot',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name # Save after each major task group
        )

        # Measure nature
        self.run_classification_task(
            task_name='measure_nature',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        # Timeline
        self.run_classification_task(
            task_name='timeline',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        # Intervention Type
        self.run_classification_task(
            task_name='intervention_type',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        # Product Categories
        self.run_classification_task(
            task_name='product_categories',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        # Broad categories

        self.run_classification_task(
            task_name='broad_categories',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        # Stated motive
        self.run_classification_task(
            task_name='stated_motive',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        # Tense Extraction
        self.run_classification_task(
            task_name='tense_extraction',
            client_name=llm_client_name,
            prompt_type='basic',
            text_column='sentence_text',
            save_path=Path(save_dir) / final_save_name
        )

        if self.processed_data.empty:
            print("No documents classified as 'reform' by the LLM. Exiting pipeline.")
            self.processed_data = None
            return

        # Save Data
        self.processed_data.to_pickle(Path(save_dir) / "all_articles_with_reform_classification.pkl")

        print(f"Step 2 completed: Documents filtered by LLM reform classification. {len(self.processed_data)} documents remain. Saved to {Path(save_dir) / llm_filtered_save_name}")

        # Run dictionary-based sentiment if path is provided
        if sentiment_csv_path:
            self.load_sentiment_dictionary(sentiment_csv_path)
            self.run_dictionary_sentiment(
                text_column='sentence_text',
                save_path=Path(save_dir) / final_save_name
            )

        print(f"Pipeline finished. Final results saved to {Path(save_dir) / final_save_name}")

In [ ]:
# Instantiate the ReformAnalyser
analyser = TradeAnalyser()

In [ ]:
# 4. Define parameters for the pipeline
input_file = "/content/test_imf_trade.csv"
text_col = 'body'
llm_client = 'openai' # Or 'openai'
llm_model = 'gpt-4o-mini' # Or 'gpt-4o', 'gpt-3.5-turbo' etc.
output_directory = 'analysis_results'
sentiment_lexicon_path = None # Set to a CSV path if you have a sentiment dictionary

print("\nRunning the full analysis pipeline...")
try:
    analyser.run_full_pipeline(
        file_path=input_file,
        text_column=text_col,
        llm_client_name=llm_client,
        llm_model_id=llm_model,
        sentiment_csv_path=sentiment_lexicon_path,
        save_dir=output_directory
    )

    # 5. Retrieve and display the final processed data
    final_results_df = analyser.get_processed_data()
    print("\n--- Final Processed Data (first 5 rows) ---")
    print(final_results_df.head())

    print(f"\nFinal processed data saved to: {Path(output_directory) / 'final_analysis_results.pkl'}")

except Exception as e:
    print(f"\nAn error occurred during pipeline execution: {e}")


Running the full analysis pipeline...
Starting the Reform Analysis Pipeline...
Loading data from /content/test_imf_trade.csv...
Data loaded. Shape: (1, 17)
Initial data loaded. Number of documents: 1
Splitting text into sentences using 'spacy' strategy...
Text splitting completed. Total sentences: 26
Data saved to 'analysis_results/filtered_by_keywords.pkl'.
Step 1: Documents filtered by keywords. 26 documents contains reforms keywords. Saved to analysis_results/filtered_by_keywords.pkl
Step 2: Classifying reforms using LLM (reform_classification)...
⚙️ Loading LLM client: openai with model 'gpt-4o-mini'...
LLM client 'openai' loaded successfully.
Running 'sentiment_analysis' with 'openai' (basic)...
```xml
<sentiment_label>moderate positive</sentiment_label>
<tone_score>3.0</tone_score>
<justification>The phrase "for a good cause" indicates a positive sentiment towards the E-Levy, suggesting it has beneficial intentions. However, the term "bittersweet medicine" introduces a slight ne

In [ ]:
analyser.processed_data.to_excel("analysis_results/final_analysis_results.xlsx")

In [ ]:
!zip -r analysis_results_5_v2.zip "analysis_results"

  adding: analysis_results/ (stored 0%)
  adding: analysis_results/filtered_by_keywords.pkl (deflated 55%)
  adding: analysis_results/filtered_by_llm.pkl (deflated 55%)
  adding: analysis_results/final_analysis_results.xlsx (deflated 4%)
  adding: analysis_results/sentences_processed.pkl (deflated 56%)
  adding: analysis_results/all_articles_with_reform_classification.pkl (deflated 55%)
  adding: analysis_results/final_analysis_results.pkl (deflated 59%)


In [ ]:
analyser.processed_data